### Passo 1 - Importar Arquivos e Bibliotecas

In [1]:
import pandas as pd
import pathlib
import smtplib
import email.message


In [2]:
# importar base de dados
emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')
#display(lojas)


### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [3]:
# incluir lojas em vendas
vendas = vendas.merge(lojas, on='ID Loja')
#display(vendas)


### Passo 3 - Salvar a planilha na pasta de backup

In [4]:
dicionario_lojas = {}
for loja in lojas['Loja']: # para cada loja na lista lojas eu quero a coluna 'Loja'
    dicionario_lojas[loja] = vendas.loc[vendas['Loja'] ==loja, :] # cria uma tabela para cada loja
#display(dicionario_lojas['Rio Mar Recife'])
#display(dicionario_lojas['Shopping Vila Velha'])

In [5]:
# Dia do indicador
dia_indicador = vendas["Data"].max() # vai pegar o ultimo dia da coluna 'Data'
#print(dia_indicador)
#print('{}/{}'.format(dia_indicador.day, dia_indicador.month))


In [6]:
# identificar se a pasta já existe
caminho_backup = pathlib.Path(r'Backup Arquivos Lojas') # Path função do pathlib que vai criar o caminho do arquivo
arquivos_pasta_backup = caminho_backup.iterdir() # iterdir() vai listar todos os arquivos dentro da pasta
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup] # name vai pegar o nome do arquivo
#print(lista_nomes_backup)
for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja # cria uma pasta com o nome da loja a "/" só pode ser usada por conta do pathlib
        nova_pasta.mkdir() # cria  a pasta

# salvar dentro pasta
    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
    local_arquivo = caminho_backup / loja /nome_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo) # trasforma o arquivo loja em excel e salva no local_arquivo


### Passo 4 - Calcular o indicador para 1 loja

In [7]:
# METAS
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

In [8]:
for loja in dicionario_lojas:
    
    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data'] == dia_indicador, :]
    # vendas_loja_dia é a vendas_loja.loc onde a coluna 'Data' é igual ao dia_indicador 
    # faturamento   
    faturamento_ano = vendas_loja['Valor Final'].sum() # soma a coluna "Valor Final"
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()
    #print(faturamento_ano)
    #print(faturamento_dia)
    # Diversidade de produtos
    qtde_produtos_ano = len(vendas_loja['Produto'].unique()) # conta a quantia de produtos unicos ".unique()" exclui os produtos duplicados
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
    #print(qtde_produtos_ano)
    #print(qtde_produtos_dia)
    # Ticket médio 
    valor_venda = vendas_loja.groupby('Código Venda').sum(numeric_only=True) # agrupa as vendas pelo Código 
    ticket_medio_ano = valor_venda['Valor Final'].mean() # tira a média dos valores finais
    #print(ticket_medio_ano)
    # tcket_medio_dia
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum(numeric_only=True)
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    #print(ticket_medio_dia)

    # ENVIAR E-MAIL GMAIL

    senha = 'trar omgq rpld ryja '
    nome = emails.loc[emails['Loja'] == loja, 'Gerente'].values[0]

    msg = email.message.EmailMessage()
    msg['From'] = "ricardogropofx@gmail.com"
    msg["Cc"] = "ricardogropofx@gmail.com"
    msg["To"] = emails.loc[emails['Loja'] == loja, 'E-mail'].values[0]
    msg["Subject"] = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'
    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    if qtde_produtos_dia >= meta_qtdeprodutos_dia:
        cor_qte_dia = 'green'
    else:
        cor_qte_dia = 'red'
    if qtde_produtos_ano >= meta_qtdeprodutos_dia:
        cor_qte_ano = 'green'
    else:
        cor_qte_ano = 'red'    
    if ticket_medio_dia >= meta_ticketmedio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'
    if ticket_medio_ano >= meta_ticketmedio_dia:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'
    corpo_email = f'''
    <p>Bom dia, {nome}</p>

    <p> O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})<strong> da <strong>Loja {loja}<strong></p>

    <table>
    <tr>
        <th>Indicador </th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
    </tr>
    
    <tr>
        <td>Faturamento</td>
        <td style="text-align: center>R$ {faturamento_dia:.2f}</td>
        <td style="text-align: center>R$ {meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color "{cor_fat_dia}">◙</font></td>
    </tr>
    
    <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center>{qtde_produtos_dia}</td>
        <td style="text-align: center>{meta_qtdeprodutos_dia}</td>
        <td style="text-align: center"><font color "{cor_qte_dia}">◙</font></td>
    </tr>
    
        <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center>R$ {ticket_medio_dia:.2f}</td>
        <td style="text-align: center>R$ {meta_ticketmedio_dia:.2f}</td>
        <td style="text-align: center"><font color "{cor_ticket_dia}">◙</font></td>
    </tr>
    </table>
    <br>
    <table>
    <tr>
        <th>Indicador </th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
    </tr>
    
    <tr>
        <td>Faturamento</td>
        <td style="text-align: center>R$ {faturamento_ano:.2f}</td>
        <td style="text-align: center>R$ {meta_faturamento_ano:.2f}</td>
        <td style="text-align: center><font color "{cor_fat_ano}">◙</font></td>
    </tr>
    
    <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center>{qtde_produtos_ano}</td>
        <td style="text-align: center>{meta_qtdeprodutos_ano}</td>
        <td style="text-align: center><font color "{cor_qte_ano}">◙</font></td>
    </tr>
    
        <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center>R$ {ticket_medio_ano:.2f}</td>
        <td style="text-align: center>R$ {meta_ticketmedio_ano:.2f}</td>
        <td style="text-align: center><font color "{cor_ticket_ano}">◙</font></td>
    </tr>
    </table>

    <p> Segue em anexo a planilha com todos os dados.</p>

    <p> Qualquer dúvida estou à disposição.</p>

    <p> Att. Ricardo</p>
    '''

    msg.add_header("Content-Type", "text/html")
    msg.set_payload(corpo_email.encode('utf-8'))

    #anexar arquivos
    attachment = open(local_arquivo, 'rb')
    msg.add_attachment(attachment.read(), maintype="application", subtype="octet-stream", filename=attachment.name)
    attachment.close()

    try:
        servidor = smtplib.SMTP('smtp.gmail.com', 587)
        servidor.starttls()
        servidor.login(msg["From"], senha)
        servidor.send_message(msg)
        print(f'Email da {loja} enviado')
    except smtplib.SMTPException as e:
        print(f'Falha ao enviar email da {loja}: {e}')
    finally:
        servidor.quit()
                




Email da Iguatemi Esplanada enviado
Email da Shopping Midway Mall enviado
Email da Norte Shopping enviado
Email da Shopping Iguatemi Fortaleza enviado
Email da Shopping União de Osasco enviado
Email da Shopping Center Interlagos enviado
Email da Rio Mar Recife enviado
Email da Salvador Shopping enviado
Email da Rio Mar Shopping Fortaleza enviado
Email da Shopping Center Leste Aricanduva enviado
Email da Ribeirão Shopping enviado
Email da Shopping Morumbi enviado
Email da Parque Dom Pedro Shopping enviado
Email da Bourbon Shopping SP enviado
Email da Palladium Shopping Curitiba enviado
Email da Passei das Águas Shopping enviado
Email da Center Shopping Uberlândia enviado
Email da Shopping Recife enviado
Email da Shopping Vila Velha enviado
Email da Shopping SP Market enviado
Email da Shopping Eldorado enviado
Email da Shopping Ibirapuera enviado
Email da Novo Shopping Ribeirão Preto enviado
Email da Iguatemi Campinas enviado
Email da Shopping Barra enviado


### Passo 6 - Automatizar todas as lojas

### Passo 7 - Criar ranking para diretoria

In [9]:
faturamento_lojas = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)
#display(faturamento_lojas_ano)  # cria o Ranking das Lojas por faturamento no ano

nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas/{}'.format(nome_arquivo)) # salva o arquivo

venda_dia = vendas.loc[vendas['Data'] == dia_indicador, :] # cria o Ranking do faturamento por Dia
faturamento_lojas_dia = venda_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)
#display(faturamento_lojas_dia)

nome_arquivo = '{}_{}_Ranking Dia.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas/{}'.format(nome_arquivo)) # salva o arquivo


### Passo 8 - Enviar e-mail para diretoria

In [10]:
for loja in dicionario_lojas:
    
    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data'] == dia_indicador, :]
    # vendas_loja_dia é a vendas_loja.loc onde a coluna 'Data' é igual ao dia_indicador 
    # faturamento   
    faturamento_ano = vendas_loja['Valor Final'].sum() # soma a coluna "Valor Final"
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()
    #print(faturamento_ano)
    #print(faturamento_dia)
    # Diversidade de produtos
    qtde_produtos_ano = len(vendas_loja['Produto'].unique()) # conta a quantia de produtos unicos ".unique()" exclui os produtos duplicados
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
    #print(qtde_produtos_ano)
    #print(qtde_produtos_dia)
    # Ticket médio 
    valor_venda = vendas_loja.groupby('Código Venda').sum(numeric_only=True) # agrupa as vendas pelo Código 
    ticket_medio_ano = valor_venda['Valor Final'].mean() # tira a média dos valores finais
    #print(ticket_medio_ano)
    # tcket_medio_dia
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum(numeric_only=True)
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    #print(ticket_medio_dia)

    # ENVIAR E-MAIL GMAIL

    senha = 'trar omgq rpld ryja '
    nome = emails.loc[emails['Loja'] == loja, 'Gerente'].values[0]

    msg = email.message.EmailMessage()
    msg['From'] = "ricardogropofx@gmail.com"
    msg["Cc"] = "ricardogropofx@gmail.com"
    msg["To"] = emails.loc[emails['Loja'] == 'Diretoria', 'E-mail'].values[0]
    msg["Subject"] = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'
    
    corpo_email = f'''Bom Dia Diretoria,
    Melhor Loja do Dia em Faturamento: Loja{faturamento_lojas_dia.index[0]}, com Faturamento R$ {faturamento_lojas_dia.iloc[0, 0]}
    Pior Loja do Dia em Faturamento: Loja{faturamento_lojas_dia.index[-1]}, com Faturamento R$ {faturamento_lojas_dia.iloc[-1, 0]}
    Melhor Loja do Ano em Faturamento: Loja{faturamento_lojas_ano.index[0]}, com faturamento R$ {faturamento_lojas_ano.iloc[0, 0]}
    Pior Loja do Ano em Faturamento: Loja{faturamento_lojas_ano.index[0]}, com faturamento R$ {faturamento_lojas_ano.iloc[-1, 0]}
    Qualquer dúvida estou à disposição.
    Att.,
    Ricardo '''

    msg.add_header("Content-Type", "text/html")
    msg.set_payload(corpo_email.encode('utf-8'))

    #anexar arquivos
    attachment = open(local_arquivo, 'rb')
    msg.add_attachment(attachment.read(), maintype="application", subtype="octet-stream", filename=attachment.name)
    attachment.close()

    servidor = smtplib.SMTP('smtp.gmail.com', 587)
    servidor.starttls()
    servidor.login(msg["From"], senha)
    servidor.send_message(msg)
    servidor.quit()
    print(f'Email da {loja} enviado')

Email da Iguatemi Esplanada enviado
Email da Shopping Midway Mall enviado
Email da Norte Shopping enviado
Email da Shopping Iguatemi Fortaleza enviado
Email da Shopping União de Osasco enviado
Email da Shopping Center Interlagos enviado
Email da Rio Mar Recife enviado
Email da Salvador Shopping enviado
Email da Rio Mar Shopping Fortaleza enviado
Email da Shopping Center Leste Aricanduva enviado
Email da Ribeirão Shopping enviado
Email da Shopping Morumbi enviado
Email da Parque Dom Pedro Shopping enviado
Email da Bourbon Shopping SP enviado
Email da Palladium Shopping Curitiba enviado
Email da Passei das Águas Shopping enviado
Email da Center Shopping Uberlândia enviado
Email da Shopping Recife enviado
Email da Shopping Vila Velha enviado
Email da Shopping SP Market enviado
Email da Shopping Eldorado enviado
Email da Shopping Ibirapuera enviado
Email da Novo Shopping Ribeirão Preto enviado
Email da Iguatemi Campinas enviado
Email da Shopping Barra enviado
